In [39]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [92]:
train_df = pd.read_csv('./train.csv', parse_dates=['Date'])
spray_df = pd.read_csv('./spray.csv', parse_dates =['Date', 'Time'])

In [93]:
weather_df = pd.read_csv('weather.csv', 
                         parse_dates = ['Date', 'Sunrise', 'Sunset'], 
                         dtype = {
                             'Depart':'Int64', 
                             'WetBulb':'Int64', 
                             'Heat':'Int64', 
                             'Cool':'Int64', 
                             'Depth':'Int64', 
                             #'SnowFall':'Int64', 
                             #'PercipTotal':'float64', 
                             'StnPressure':'float64', 
                             'SeaLevel':'float64', 
                             'AvgSpeed':'float64'}, 
                         na_values = ['M','-'])

In [75]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      2944 non-null   int64         
 1   Date         2944 non-null   datetime64[ns]
 2   Tmax         2944 non-null   int64         
 3   Tmin         2944 non-null   int64         
 4   Tavg         2933 non-null   float64       
 5   Depart       1472 non-null   Int64         
 6   DewPoint     2944 non-null   int64         
 7   WetBulb      2940 non-null   Int64         
 8   Heat         2933 non-null   Int64         
 9   Cool         2933 non-null   Int64         
 10  Sunrise      1472 non-null   object        
 11  Sunset       1472 non-null   object        
 12  CodeSum      2944 non-null   object        
 13  Depth        1472 non-null   Int64         
 14  Water1       0 non-null      float64       
 15  SnowFall     1472 non-null   object        
 16  Precip

In [78]:
weather_df.isnull().sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg             11
Depart         1472
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise        1472
Sunset         1472
CodeSum           0
Depth          1472
Water1         2944
SnowFall       1472
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [94]:
weather_df['PrecipTotal'] = weather_df['PrecipTotal'].map(lambda x: np.nan if x == '  T' else x)
weather_df['SnowFall'] = weather_df['SnowFall'].map(lambda x: np.nan if x == '  T' else x)

In [ ]:
weather_df[weather_df['PrecipTotal'] == '  T'].count()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [87]:
weather_df[weather_df['SnowFall'] == '  T'].count()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [88]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      2944 non-null   int64         
 1   Date         2944 non-null   datetime64[ns]
 2   Tmax         2944 non-null   int64         
 3   Tmin         2944 non-null   int64         
 4   Tavg         2933 non-null   float64       
 5   Depart       1472 non-null   Int64         
 6   DewPoint     2944 non-null   int64         
 7   WetBulb      2940 non-null   Int64         
 8   Heat         2933 non-null   Int64         
 9   Cool         2933 non-null   Int64         
 10  Sunrise      1472 non-null   object        
 11  Sunset       1472 non-null   object        
 12  CodeSum      2944 non-null   object        
 13  Depth        1472 non-null   Int64         
 14  Water1       0 non-null      float64       
 15  SnowFall     1460 non-null   object        
 16  Precip

In [95]:
weather_df[['PrecipTotal', 'SnowFall']] = weather_df[['PrecipTotal', 'SnowFall']].apply(pd.to_numeric)

In [96]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Station      2944 non-null   int64         
 1   Date         2944 non-null   datetime64[ns]
 2   Tmax         2944 non-null   int64         
 3   Tmin         2944 non-null   int64         
 4   Tavg         2933 non-null   float64       
 5   Depart       1472 non-null   Int64         
 6   DewPoint     2944 non-null   int64         
 7   WetBulb      2940 non-null   Int64         
 8   Heat         2933 non-null   Int64         
 9   Cool         2933 non-null   Int64         
 10  Sunrise      1472 non-null   object        
 11  Sunset       1472 non-null   object        
 12  CodeSum      2944 non-null   object        
 13  Depth        1472 non-null   Int64         
 14  Water1       0 non-null      float64       
 15  SnowFall     1460 non-null   float64       
 16  Precip

In [97]:
def compare_stn(df):
    ori_coord = (df['Latitude'], df['Longitude'])
    dist_to_stn1 = geodesic(ori_coord, (41.995, -87.993)).kilometers # Comparing distance to Station 1
    dist_to_stn2 = geodesic(ori_coord, (41.786, -87.752)).kilometers #Comparing distance to station 2
    if dist_to_stn1 <= dist_to_stn1:
        return 1
    else:
        return 2

In [98]:
# applying function to check if locations are closer to station 1 and 2 and returning the stations accordingly
train_df['Station'] = train_df.apply(compare_stn, axis = 1)

In [99]:
train_df.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Station
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,1
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,1
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,1
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,1
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,1


In [100]:
# merging train and weather data based on date and station
df_merge = pd.merge(train_df, weather_df, on = ['Date', 'Station'], how = 'left')

In [101]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10506 entries, 0 to 10505
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    10506 non-null  datetime64[ns]
 1   Address                 10506 non-null  object        
 2   Species                 10506 non-null  object        
 3   Block                   10506 non-null  int64         
 4   Street                  10506 non-null  object        
 5   Trap                    10506 non-null  object        
 6   AddressNumberAndStreet  10506 non-null  object        
 7   Latitude                10506 non-null  float64       
 8   Longitude               10506 non-null  float64       
 9   AddressAccuracy         10506 non-null  int64         
 10  NumMosquitos            10506 non-null  int64         
 11  WnvPresent              10506 non-null  int64         
 12  Station                 10506 non-null  int64 

In [102]:
df_merge_overall = pd.merge(df_merge, spray_df, on = ['Latitude', 'Longitude', 'Date'], how = 'left')

In [103]:
df_merge_overall.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10506 entries, 0 to 10505
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    10506 non-null  datetime64[ns]
 1   Address                 10506 non-null  object        
 2   Species                 10506 non-null  object        
 3   Block                   10506 non-null  int64         
 4   Street                  10506 non-null  object        
 5   Trap                    10506 non-null  object        
 6   AddressNumberAndStreet  10506 non-null  object        
 7   Latitude                10506 non-null  float64       
 8   Longitude               10506 non-null  float64       
 9   AddressAccuracy         10506 non-null  int64         
 10  NumMosquitos            10506 non-null  int64         
 11  WnvPresent              10506 non-null  int64         
 12  Station                 10506 non-null  int64 

In [104]:
df_merge_overall.to_csv('merged.csv')

0   Date:10506 non-null  datetime64[ns] -- Date of WNV Test
 1   Address:10506 non-null  object -- Location of the Trap
 2   Species:10506 non-null  object --Species of Mosquitos
 3   Block:10506 non-null  int64 --Block Num in Address
 4   Street:10506 non-null  object -- Street Name in Address
 5   Trap:10506 non-null  object -- id of trap
 6   AddressNumberAndStreet:10506 non-null  object --Adress    
 7   Latitude:10506 non-null  float64 --Latitude of Adress
 8   Longitude:10506 non-null  float64 --longitude of Adress
 9   AddressAccuracy:10506 non-null  int64--Accuracy of address 
 10  NumMosquitos:10506 non-null  int64-- Num of Mosquitoes caught     
 11  WnvPresent:10506 non-null  int64 --Presence of WNV
 12  Station:10506 non-null  int64 -- Nearest Weather Station
 13  Tmax:10506 non-null  int64 -- Max temp in Fahrenheit
 14  Tmin:10506 non-null  int64 --Min temp in Fahrenheit
 15  Tavg :10506 non-null  float64 -- Avg temp in Fahrenheit
 16  Depart :10506 non-null  int64 --Temp Departure from normal Fahrenheit  
 17  DewPoint:10506 non-null  int64 --Average Dewpoint temp  in Fahrenheit    
 18  WetBulb:10506 non-null  int64-- Average wet bulb temp in Fahrenheit    
 19  Heat:10506 non-null  int64--Abs temp diff of Tavg from base temp (65 deg) if Tavg <  65 deg
 20  Cool:10506 non-null  int64  -- Abs temp diff of Tavg from base temp (65 deg) if Tavg >  65 deg     
 21  Sunrise: 10506 non-null  object --  Time of Sunrise     
 22  Sunset :10506 non-null  object --  Time of Sunset     
 23  CodeSum: 10506 non-null  object – Weather Phenomena      
 24  Depth: 10506 non-null  int64  -- Snow/ice in inches     
 25  Water1:10506 non-null  float64 --   Water equivalent depth    
 26  SnowFall: 10506 non-null  float64 --   snowfall in inches and tenths   
 27  PrecipTotal: 10506 non-null  float64 -- Rainfall and melted snow in inches and hundredths      
 28  StnPressure :10506 non-null  float64 -- Average Station Pressure       
 29  SeaLevel :10506 non-null  float64 – Average sea level pressure      
 30  ResultSpeed :10506 non-null  float64 --   resultant wind speeds in miles/hr    
 31  ResultDir:10506 non-null  int64 --  resultant wind direction in deg       
 32  AvgSpeed :10506 non-null  float64  -- average wind sped in miles/h     
 33  Time:  0 non-null      datetime64[ns] – Time of Spray
